In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import folium

In [2]:
taxi = pd.read_csv("taxi_0516.csv", index_col = 0)
col_ind = [0] + list(range(taxi.shape[1]))[5:]

Просуммируем общее количество поездок такси из каждой географической зоны и посчитаем количество ячеек, из которых в мае не было совершено ни одной поездки.

In [3]:
print("Всего зон:", taxi.shape[0])
ind_to_drop = []
zero_zones_count = 0
for ind in range(taxi.iloc[:, 5:].shape[0]):
    if np.array(taxi.iloc[:, 5:])[ind].sum() == 0:
        zero_zones_count += 1
        ind_to_drop.append(ind)
print("Зон без поездок замай 2016 года:", zero_zones_count)
taxi.drop(index = ind_to_drop, inplace = True, errors = 'ignore')

Всего зон: 2500
Зон без поездок замай 2016 года: 1283


In [4]:
print("Новый размер фрейма:", taxi.shape)

Новый размер фрейма: (1217, 749)


Нарисуем статическую карту Нью-Йорка. Поставим на карте точку там, где находится Эмпайр-Стейт-Билдинг.


In [5]:
ESB_coords = [40.748817, -73.985428]
title = 'ESB coordinates'
#Сделаем карту с пока с фиксированным зумом, чтобы нарисовать на ней кружочек
my_map = folium.Map(location = ESB_coords, png_enabled = False,
                   zoom_start = 12, min_zoom = 12, max_zoom = 12)
#рисуем кружочек, размер - дело вкуса
folium.Circle(ESB_coords, 150, fill = True).add_child(folium.Popup('Empire State Building')).add_to(my_map)
tooltip = 'Empire State Building is here!'
folium.Marker(ESB_coords, popup = '<div style="font-size: 10pt"><b>%s</b></div>' % title,
              tooltip = tooltip).add_to(my_map)
my_map.save('map_1.html')

In [6]:
my_map

Поверх статической карты Нью-Йорка визуализируем данные о поездках из каждой ячейки так, чтобы цветовая шкала, в которую окрашивается каждая ячейка, показывала суммарное количество поездок такси из неё.

In [7]:
# Агрегируем данные
trips = np.array([el.sum() for el in np.array(taxi.iloc[:, 5:])])
labels = np.array(taxi.iloc[:, :5])

In [8]:
from colour import Color
color_1 = 'Green'
color_2 = 'Blue'
color_1 = Color(color_1)
grad = list(color_1.range_to(Color(color_2), int(trips.max())))

my_gradient_map = folium.Map(location=  [40.75, -74.00], 
                            zoom_start = 10, 
                            png_enabled = False)
for i in range(taxi.shape[0]):
    
    _, W, E, S, N = np.array(taxi.iloc[:, :5])[i]
    left_down = (S, W)
    left_up = (N, W)
    right_up = (N, E)
    right_down = (S, E)
    rectangle = [left_down, left_up, right_up, right_down]
    folium.vector_layers.Rectangle(rectangle, popup = None, tooltip = str(int(labels[i][0])), 
                                   fill = True, fill_color = grad[int(trips[i] - 1)].hex_l, 
                                   fill_opacity = 0.5,
                                   stroke = True, color = 'black',
                                   weight = 1, opacity = 0.1).add_to(my_gradient_map)

my_gradient_map.save('map_2.html')

In [9]:
my_gradient_map
# видно, что такси могут ездить по воде, но такого не может быть!

Вставим интерактивную карту Нью-Йорка — такую, которую можно прокручивать и увеличивать. Поставим метку там, где находится статуя Свободы.

In [10]:
LS_coords = [40.689247, -74.044502]
title = 'Statue Of Liberty'
my_freedom_map = folium.Map(location = LS_coords, 
               zoom_start = 12, min_zoom = 10, png_enabled = False)

folium.Circle(LS_coords, 150, fill=True, color=Color('Yellow').hex_l).add_child(
    folium.Popup('Statue Of Liberty')).add_to(my_freedom_map)
tooltip = 'The Liberty Monument is here!'
folium.Marker(LS_coords, popup = '<div style="font-size: 10pt"><b>%s</b></div>' % title, 
              tooltip = tooltip).add_to(my_freedom_map)
my_freedom_map.save('map_3.html')

In [11]:
my_freedom_map

Нарисуем на интерактивной карте Нью-Йорка ячейки так, чтобы их цвет показывал среднее за месяц количество поездок такси в час из этой зоны.

Чтобы не выбирать из всех 2500 ячеек вручную, отфильтруем ячейки, из которых в мае совершается в среднем меньше 5 поездок в час. Посчитайем количество оставшихся. Проверим на карте, что среди этих ячеек нет таких, из которых поездки на самом деле невозможны.

In [5]:
taxi = taxi.reset_index(drop = True)

In [6]:
#отфильтруем ячейки с количеством поездок меньше пяти в час
tmp = np.array(taxi.iloc[:, 5:])
ind_to_drop_ = []
for ind in range(tmp.shape[0]):
    arr = tmp[ind]
    if np.mean(arr) < 5:
        ind_to_drop_.append(ind)
        
print("Количество ячеек с меньшим замай 2016 года средним числом поездок меньше пяти за час:", len(ind_to_drop_))
taxi.drop(index = ind_to_drop_, inplace = True, errors = 'ignore')
print("Что осталось:", taxi.shape[0])

Количество ячеек с меньшим замай 2016 года средним числом поездок меньше пяти за час: 1115
Что осталось: 102


In [7]:
taxi.shape
taxi = taxi.reset_index(drop = True)

In [10]:
# отобразим карту после фильтрации
# Агрегируем данные

trips = np.array([el.sum() for el in np.array(taxi.iloc[:, 5:])])
labels = np.array(taxi.iloc[:, :5])

color_1 = 'Yellow'
color_2 = 'Red'
color_1 = Color(color_1)
grad = list(color_1.range_to(Color(color_2), int(trips.max())))

my_gradient_map_2 = folium.Map(location=  [40.75, -74.00], 
                            zoom_start = 10, 
                            png_enabled = False)
for i in range(taxi.shape[0]):
    
    _, W, E, S, N = np.array(taxi.iloc[:, :5])[i]
    left_down = (S, W)
    left_up = (N, W)
    right_up = (N, E)
    right_down = (S, E)
    rectangle = [left_down, left_up, right_up, right_down]
    folium.vector_layers.Rectangle(rectangle, popup = None, tooltip = str(int(labels[i][0])), 
                                   fill = True, fill_color = grad[int(trips[i] - 1)].hex_l, 
                                   fill_opacity = 0.5,
                                   stroke = True, color = 'black',
                                   weight = 1, opacity = 0.1).add_to(my_gradient_map_2)

my_gradient_map_2.save('map_4.html')

In [11]:
#вроде таких ячеек больше нету
my_gradient_map_2